In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 83.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.1 MB/

In [15]:
import os
import numpy as np
import gradio as gr
import librosa
import librosa.feature as Aud
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pickle

Models = ["Logistic Reg.","Random Forest"]

loaded_scaler = pickle.load(open('/scaler_regression.sav', 'rb'))
loaded_model_Reg = pickle.load(open('/finalized_model_regression.sav', 'rb'))
loaded_model_Forest = pickle.load(open('/finalized_model_RandomF.sav', 'rb'))

def build_record(sr, x):
  record = {}
  chroma = Aud.chroma_stft(y=x, sr=sr)
  rms = Aud.rms(y=x)
  cent = Aud.spectral_centroid(y=x, sr=sr)
  spec_bw = Aud.spectral_bandwidth(y=x, sr=sr)
  rolloff = Aud.spectral_rolloff(y=x, sr=sr)
  zero_cross = Aud.zero_crossing_rate(x)
  onset_env = librosa.onset.onset_strength(y=x, sr=sr)
  tempo = Aud.tempo(onset_envelope=onset_env, sr=sr)

  record['chroma_stft_mean'] = np.mean(chroma)
  record['chroma_stft_var'] = np.var(chroma)
  record['rms_mean'] = np.mean(rms)
  record['rms_var'] = np.var(rms)
  record['spectral_centroid_mean'] = np.mean(cent)
  record['spectral_centroid_var'] = np.var(cent)
  record['spectral_bandwidth_mean'] = np.mean(spec_bw)
  record['spectral_bandwidth_var'] = np.var(spec_bw)
  record['rolloff_mean'] = np.mean(rolloff)
  record['rolloff_var'] = np.var(rolloff)
  record['zero_crossing_rate_mean'] = np.mean(zero_cross)
  record['zero_crossing_rate_var'] = np.var(zero_cross)
  record['tempo'] = tempo

  mfccs = Aud.mfcc(y = x, sr = sr)
  for i in range(mfccs.shape[0]):
    key_m = 'mfcc' + str(i+1) + '_mean'
    key_v = 'mfcc' + str(i+1) + '_var'
    record[key_m] = np.mean(mfccs[i,:])
    record[key_v] = np.var(mfccs[i,:])

  return record

def Logistic_Regression(X):
  # load the model from disk
  scaled_vals = loaded_scaler.transform(X)
  result = loaded_model_Reg.predict(scaled_vals)
  return result

def Random_Forest(X):
  # load the model from disk
  result = loaded_model_Forest.predict(X)
  return result

def main_note(audio,model):
    #sr, x = audio
    x , sr = librosa.load(audio, sr=None)
    #print(type(sr),type(x))
    registro = build_record(sr, x)
    df = pd.DataFrame(registro)
    df = df.values
    out_ = "Nothing"

    if model == 0:
      out_ = Logistic_Regression(df)
    elif model == 1:
      out_ = Random_Forest(df)

    return out_[0]


demo = gr.Interface(
    main_note,
    [
        gr.Audio(type='filepath'),
        gr.Dropdown(Models, type="index")
    ],
    #gr.Label(label='Info de la pista', num_top_classes = 6),
    gr.Textbox(value="", label="Género de la pista", interactive=False),
    examples=[
        [os.path.join('/',"Audio/blues.00014.wav")],
        [os.path.join('/',"Audio/country.00011.wav")],
        [os.path.join('/',"Audio/metal.00017.wav")]
    ],
)

if __name__ == "__main__":
    demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://72cf489b84d8577360.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://72cf489b84d8577360.gradio.live
